In [1]:
cd /home/ubuntu/fedatk_unl_tj/

/home/ubuntu/fedatk_unl_tj


In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd
import gc

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from utils.util_notebooks import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *

/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:
setting, num_user = "FedAvg_adv", 40
exp = "cifar10"

try: # Skip loading if already loaded
    aggregator
except:
    aggregator, clients, args_ = set_args(setting, num_user,  experiment = exp) # Indicate dataset here

# Load models for FAT and FedAvg
save_path_FAT = '/home/ubuntu/fedatk_unl_tj/weights/cifar10/231031_FAT150round/FAT/'
save_path_FedAvg =  '/home/ubuntu/fedatk_unl_tj/weights/cifar10/230922_baseline_train/fedavg/'

model_FAT = copy.deepcopy(import_model_weights(num_user, setting, save_path_FAT, aggregator, args_)[0])
model_Fedavg = import_model_weights(num_user, setting, save_path_FedAvg, aggregator, args_)[0]

# del aggregator, clients, args_
# torch.cuda.empty_cache()


==> Clients initialization..
===> Building data iterators..


 49%|████▉     | 39/80 [00:00<00:00, 118.02it/s]


===> Initializing clients..


  0%|          | 0/40 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 40/40 [00:16<00:00,  2.37it/s]
/home/ubuntu/fedatk_unl_tj/aggregator.py:289: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious p

In [4]:
# Obtain parameters for each layer
params_FAT = model_FAT.state_dict()
params_FedAvg = model_Fedavg.state_dict()

# Just take the values of weights and bias for the model
desired_keys = [key for key in params_FAT.keys() if 'weight' in key or 'bias' in key]

In [5]:
num_aru = 1
num_rounds = 1
eps_train = 4.5
eps_attack = 4
weight_2 = 1
step_size = 0.01
steps = 10

In [6]:
# Perform rounds of FAT on FedAvg model
aggregator.load_state(dir_path=save_path_FAT)
aggregator.update_clients()
aggregator.op = None
weight_scale_2 = 1

# UNL_mix(aggregator, adv_id=[0], model_inject=model_Fedavg, weight_scale_2 = weight_scale_2, keys=desired_keys, aggregation_op = None)

# model_overfit = pull_model_from_agg(aggregator)
# model_overfit.eval()
# acc, adv_acc = get_adv_acc(aggregator, model_overfit, eps=eps_attack, step_size = step_size, steps = steps)
# print("Test acc: ", np.mean(acc), " (", np.std(acc),") ", "adv acc: ", np.mean(adv_acc)," (", np.std(adv_acc),") ")

In [47]:
# def calc_atk_model_debug(model_inject, model_global, keys, weight_scale, weight_scale_2):
#     with torch.no_grad():
#         atk_model = copy.deepcopy(model_global)
#         inject_state_dict = model_inject.state_dict(keep_vars=False)
#         global_state_dict = model_global.state_dict(keep_vars=False)
#         return_state_dict = atk_model.state_dict(keep_vars=False)
#         total_weight = weight_scale * weight_scale_2

#         for key in keys:
#             diff = inject_state_dict[key].data.clone() - global_state_dict[key].data.clone()
#             return_state_dict[key].data = total_weight * diff + global_state_dict[key].data.clone()

#         atk_model.load_state_dict(return_state_dict)
#         return atk_model

def calc_atk_model_debug(model_inject, model_global, keys, weight_scale, weight_scale_2):

    atk_model = copy.deepcopy(model_inject)
    inject_state_dict = model_inject.state_dict(keep_vars=False)
    global_state_dict = model_global.state_dict(keep_vars=False)
    return_state_dict = atk_model.state_dict(keep_vars=False)
    total_weight = weight_scale * weight_scale_2

    # for key in keys:
    for key in inject_state_dict.keys():
        print(key, "total")
        diff = inject_state_dict[key].data.clone() - global_state_dict[key].data.clone()
        return_state_dict[key].data = total_weight * diff + global_state_dict[key].data.clone()

    atk_model.load_state_dict(return_state_dict)

    return atk_model

def average_learners_inject(
        learners,
        target_learner,
        weights=None,
        average_params=True,
        average_gradients=False, 
        desired_keys = None
        ):
    """
    Compute the average of a list of learners_ensemble and store it into learner

    :param learners:
    :type learners: List[Learner]
    :param target_learner:
    :type target_learner: Learner
    :param weights: tensor of the same size as learners_ensemble, having values between 0 and 1, and summing to 1,
                    if None, uniform learners_weights are used
    :param average_params: if set to true the parameters are averaged; default is True
    :param average_gradients: if set to true the gradient are also averaged; default is False
    :type weights: torch.Tensor

    """
    if not average_params and not average_gradients:
        return

    if weights is None:
        n_learners = len(learners)
        weights = (1 / n_learners) * torch.ones(n_learners, device=learners[0].device)

    else:
        weights = weights.to(learners[0].device)

    target_state_dict = target_learner.model.state_dict(keep_vars=True)
    if desired_keys is None:
        desired_keys = [key for key in target_state_dict.keys() if 'weight' in key or 'bias' in key]

    for key in desired_keys:

        if target_state_dict[key].data.dtype == torch.float32:

            if average_params:
                target_state_dict[key].data.fill_(0.)

            if average_gradients:
                target_state_dict[key].grad = target_state_dict[key].data.clone()
                target_state_dict[key].grad.data.fill_(0.)

            for learner_id, learner in enumerate(learners):
                state_dict = learner.model.state_dict(keep_vars=True)

                if average_params:
                    target_state_dict[key].data += weights[learner_id] * state_dict[key].data.clone()

                if average_gradients:
                    if state_dict[key].grad is not None:
                        target_state_dict[key].grad += weights[learner_id] * state_dict[key].grad.clone()
                    elif state_dict[key].requires_grad:
                        warnings.warn(
                            "trying to average_gradients before back propagation,"
                            " you should set `average_gradients=False`."
                        )

        else:
            # tracked batches
            target_state_dict[key].data.fill_(0)
            for learner_id, learner in enumerate(learners):
                state_dict = learner.model.state_dict()
                target_state_dict[key].data += state_dict[key].data.clone()


aggregator.load_state(dir_path=save_path_FAT)
aggregator.update_clients()
aggregator.op = None

adv_id = [0]
weight_scale =  1/aggregator.clients_weights # np.ones(aggregator.clients_weights.shape)#
model_global = aggregator.global_learners_ensemble[0].model
model_inject = model_Fedavg
aggregation_op = None
keys = desired_keys

# Based on aggregation methods change weight scale
if aggregation_op in ['median', 'krum', 'median_sublayers']:# == "median" or aggregation_op == "krum":
    weight_scale = np.ones(weight_scale.shape)

# # Give adversarial clients boosted models and train regular clients 1 round
benign_id = list(range(len(aggregator.clients)))

for a_id in adv_id:
    benign_id.remove(a_id)
    temp_atk_model = calc_atk_model_debug(model_inject, model_global, keys, weight_scale[a_id], weight_scale_2)
    # aggregator.clients[a_id].learners_ensemble[0].model.cpu()
    # del aggregator.clients[a_id].learners_ensemble[0].model
    # aggregator.clients[a_id].learners_ensemble[0].model = temp_atk_model
    # # del temp_atk_model
    # gc.collect()
    # torch.cuda.empty_cache()


# for c_id in benign_id:
#     aggregator.clients[c_id].step()

# # Aggregate model and download
# for learner_id, learner in enumerate(aggregator.global_learners_ensemble):
#     learners = [client.learners_ensemble[learner_id] for client in aggregator.clients]
#     if aggregation_op is None:
#         average_learners_inject(learners, learner, weights=aggregator.clients_weights, desired_keys = desired_keys)
#         print("averaging")

features.0.0.weight total
features.0.1.weight total
features.0.1.bias total
features.0.1.running_mean total
features.0.1.running_var total
features.0.1.num_batches_tracked total
features.1.conv.0.0.weight total
features.1.conv.0.1.weight total
features.1.conv.0.1.bias total
features.1.conv.0.1.running_mean total
features.1.conv.0.1.running_var total
features.1.conv.0.1.num_batches_tracked total
features.1.conv.1.weight total
features.1.conv.2.weight total
features.1.conv.2.bias total
features.1.conv.2.running_mean total
features.1.conv.2.running_var total
features.1.conv.2.num_batches_tracked total
features.2.conv.0.0.weight total
features.2.conv.0.1.weight total
features.2.conv.0.1.bias total
features.2.conv.0.1.running_mean total
features.2.conv.0.1.running_var total
features.2.conv.0.1.num_batches_tracked total
features.2.conv.1.0.weight total
features.2.conv.1.1.weight total
features.2.conv.1.1.bias total
features.2.conv.1.1.running_mean total
features.2.conv.1.1.running_var total


In [60]:
a = 0.0088
params_atk = temp_atk_model.state_dict()
# first make the model with empty weights
new_model = copy.deepcopy(model_FAT)
new_model.eval()
new_weight_dict = copy.deepcopy(params_FAT)
for key in new_weight_dict.keys():
    print(key)
    new_weight_dict[key] = a*params_atk[key] + (1-a)*params_FAT[key] 
new_model.load_state_dict(new_weight_dict)



features.0.0.weight
features.0.1.weight
features.0.1.bias
features.0.1.running_mean
features.0.1.running_var
features.0.1.num_batches_tracked
features.1.conv.0.0.weight
features.1.conv.0.1.weight
features.1.conv.0.1.bias
features.1.conv.0.1.running_mean
features.1.conv.0.1.running_var
features.1.conv.0.1.num_batches_tracked
features.1.conv.1.weight
features.1.conv.2.weight
features.1.conv.2.bias
features.1.conv.2.running_mean
features.1.conv.2.running_var
features.1.conv.2.num_batches_tracked
features.2.conv.0.0.weight
features.2.conv.0.1.weight
features.2.conv.0.1.bias
features.2.conv.0.1.running_mean
features.2.conv.0.1.running_var
features.2.conv.0.1.num_batches_tracked
features.2.conv.1.0.weight
features.2.conv.1.1.weight
features.2.conv.1.1.bias
features.2.conv.1.1.running_mean
features.2.conv.1.1.running_var
features.2.conv.1.1.num_batches_tracked
features.2.conv.2.weight
features.2.conv.3.weight
features.2.conv.3.bias
features.2.conv.3.running_mean
features.2.conv.3.running_var


<All keys matched successfully>

In [61]:
weight_scale[a_id]

tensor(113.6804)

In [62]:

model_global = aggregator.global_learners_ensemble[0].model
model_client = aggregator.clients[1].learners_ensemble[0].model

acc, adv_acc = get_adv_acc(aggregator, new_model, eps=eps_attack, step_size = step_size, steps = steps)
print("Test acc: ", np.mean(acc), " (", np.std(acc),") ", "adv acc: ", np.mean(adv_acc)," (", np.std(adv_acc),") ")

Test acc:  0.8549999952316284  ( 0.0646988174655441 )  adv acc:  0.010312500153668224  ( 0.009636965109295265 ) 


In [58]:
model_global.state_dict()['features.17.conv.3.running_mean']

tensor([-3.6040e-02, -1.7314e-02, -2.9185e-02,  3.6448e-02, -4.7945e-02,
        -3.2198e-02, -1.2044e-01,  9.3056e-02, -1.2594e-01, -3.4998e-03,
        -1.0203e-01, -2.3453e-03,  9.0848e-02, -1.0762e-01,  7.5059e-02,
         3.4159e-02, -6.5809e-02, -9.1424e-02,  5.9271e-02, -1.6213e-01,
        -3.8093e-02,  1.7239e-01, -3.9066e-02, -7.3285e-03,  3.5023e-02,
        -9.2273e-02,  1.6514e-01, -7.8947e-02, -1.5671e-01, -8.0183e-02,
         1.4583e-01,  3.8248e-02, -3.7716e-02, -4.0461e-02,  4.6975e-02,
         1.4656e-01, -6.9432e-02,  9.6042e-02, -1.0139e-02, -1.0109e-01,
        -2.0836e-01,  1.0307e-01, -7.5671e-02, -8.8814e-03, -1.2926e-02,
        -1.5424e-01, -9.2713e-03,  1.1414e-01, -6.6919e-02, -1.8199e-01,
         7.6220e-02, -7.8709e-02,  6.3101e-02,  4.2614e-02, -6.2896e-03,
        -6.1512e-02,  6.4384e-02,  6.9918e-02, -2.3017e-02, -1.4033e-01,
        -1.1360e-02,  1.3197e-02, -4.6689e-03,  1.7533e-02, -1.0762e-01,
        -4.3535e-02, -2.8164e-01,  5.6919e-02,  3.3

In [59]:
temp_atk_model.state_dict()['features.17.conv.3.running_mean']

tensor([ 1.2566e+00,  1.7568e-01,  5.3032e-01,  4.8528e-01,  1.4230e+00,
        -4.8630e-01,  2.6665e+00, -1.7887e+00, -1.9969e+00, -2.3407e+00,
        -2.7083e+00, -1.6274e+00,  2.9390e+00, -2.1259e-01,  3.3210e+00,
         1.2216e+00, -2.2020e-01,  5.0330e-01, -5.3409e-01,  1.2406e+00,
         3.9915e-02, -3.1031e-01,  1.8286e+00, -8.7779e-01,  7.3021e-01,
         1.5077e+00,  1.7277e-01,  5.6840e-01,  1.7641e+00,  2.4147e+00,
         3.3031e-01,  1.0210e+00, -7.3006e-01, -1.5028e-01, -2.4561e+00,
         6.0343e-01,  2.7003e+00,  1.3169e+00,  7.4770e-01,  5.8609e-01,
         1.7782e+00, -1.1555e+00,  2.4197e+00, -8.3227e-01,  1.6366e-01,
         7.0821e-01,  1.6835e+00, -1.4294e+00, -1.6206e+00,  2.8456e+00,
         1.4537e+00,  1.2339e+00,  7.5515e-01, -8.6303e-01, -2.6035e+00,
        -1.3260e+00, -2.5312e+00, -5.6667e-01,  1.9192e+00, -1.2794e-01,
        -2.0182e-01,  1.3990e+00, -1.9876e+00,  8.8675e-01,  2.3395e+00,
        -5.6239e-02, -9.2445e-01,  1.1243e+00, -4.7